In [1]:
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.linear_model import LinearRegression

from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import manifold, decomposition, linear_model, ensemble, neighbors, cross_validation
import sklearn
from sklearn import pipeline, preprocessing, feature_extraction
import xgboost as xgb
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
import time
import datetime
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, auc, roc_curve
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import sparse
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
import scipy
from sklearn.preprocessing import StandardScaler

%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [47]:
len(m_svd)

2345796

In [41]:
len(X_test)

2345796

In [42]:
404290+2345796

2750086

In [2]:
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

y_train = pd.DataFrame(X_train['is_duplicate'])
y_train.columns = ['y']
del X_train['is_duplicate']

skf = StratifiedKFold(5, shuffle=True, random_state=0)
folds = []
for itr, ite in skf.split(X_train, y_train.y):
    folds += [[itr, ite]]

In [3]:
X_train = pd.read_csv("data/train_inter.csv")

X_train['question1'] = X_train['question1'].apply(lambda x: str(x))
X_train['question2'] = X_train['question2'].apply(lambda x: str(x))

X_train['question1'] = X_train['question1'].apply(lambda x: x.lower().split())
X_train['question2'] = X_train['question2'].apply(lambda x: x.lower().split())



In [4]:
X_test = pd.read_csv("data/test_inter.csv")

X_test['question1'] = X_test['question1'].apply(lambda x: str(x))
X_test['question2'] = X_test['question2'].apply(lambda x: str(x))

X_test['question1'] = X_test['question1'].apply(lambda x: x.lower().split())
X_test['question2'] = X_test['question2'].apply(lambda x: x.lower().split())

In [5]:
X = map(lambda x: ' '.join(x), X_train['question1'])
X1 = map(lambda x: ' '.join(x), X_train['question2'])
Y = map(lambda x: ' '.join(x), X_test['question1'])
Y1 = map(lambda x: ' '.join(x), X_test['question2'])
hw = HashingVectorizer(n_features=200).fit(X_train['question1'] + X_train['question2'])


In [6]:
X = pd.DataFrame(hw.transform(X).todense())
X1 = pd.DataFrame(hw.transform(X1).todense())
Y = pd.DataFrame(hw.transform(Y).todense())
Y1 = pd.DataFrame(hw.transform(Y1).todense())

In [7]:
tfidf = feature_extraction.text.TfidfTransformer()
tfidf.fit(X+X1)

X = pd.DataFrame(tfidf.transform(X).todense())
X1 = pd.DataFrame(tfidf.transform(X1).todense())
Y = pd.DataFrame(tfidf.transform(Y).todense())
Y1 = pd.DataFrame(tfidf.transform(Y1).todense())

In [8]:
X_train = pd.concat([X_train, X], axis=1, join_axes=[X_train.index])
del X
X_train = pd.concat([X_train, X1], axis=1, join_axes=[X_train.index])
del X1
X_test = pd.concat([X_test, Y], axis=1, join_axes=[X_test.index])
del Y
X_test = pd.concat([X_test, Y1], axis=1, join_axes=[X_test.index])
del Y1

In [9]:
cores_dict = pd.read_csv("data/question_max_kcores.csv", index_col="qid").to_dict()["max_kcore"]
def gen_qid1_max_kcore(row):
    return cores_dict[row["qid1"]]
def gen_qid2_max_kcore(row):
    return cores_dict[row["qid2"]]

#def gen_max_kcore(row):
#    return max(row["qid1_max_kcore"], row["qid2_max_kcore"])

X_train["qid1_max_kcore"] = X_train.apply(gen_qid1_max_kcore, axis=1)
#X_test["qid1_max_kcore"] = X_test.apply(gen_qid1_max_kcore, axis=1)
X_train["qid2_max_kcore"] = X_train.apply(gen_qid2_max_kcore, axis=1)
#X_test["qid2_max_kcore"] = X_test.apply(gen_qid2_max_kcore, axis=1)

In [10]:
q_test = pd.read_csv("data/kcore_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [27]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,192,193,194,195,196,197,198,199,qid1_max_kcore,qid2_max_kcore
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,-0.164904,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [11]:
del X_train['question1']
del X_train['question2']
del X_train['qid1']
del X_train['qid2']
del X_train['id']

del X_test['question1']
del X_test['question2']
del X_test['test_id']

In [47]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,21,22,23,24,25,26,27,28,29,lstm
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,-0.03559,0.150354,-0.032531,-0.086795,0.057361,0.111245,-0.033171,0.066494,0.010014,0.057908
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,-0.164904,0.0,0.0,...,-0.03081,0.003312,-0.046230,-0.044494,-0.004791,0.072093,-0.169035,-0.086499,0.006689,0.421167


In [28]:
X_train.ix[:2, 202:402]

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.119495,...,0.0,0.0,0.325385,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.131759,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,-0.167796,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
X_train.ix[:2, 2:202]

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.113341,...,0.0,0.0,0.308628,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.165352,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.209907,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.380265,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.186087,0.0,0.0,0.0


In [30]:
#tmp = abs(X_train.ix[:, 2:202] * X_train.ix[:, -200:])
#X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])
tmp = abs(X_train.ix[:, 2:202] - X_train.ix[:, 202:402])
X_train = pd.concat([X_train, tmp], axis=1, join_axes=[X_train.index])

#tmp = abs(X_test.ix[:, 2:202] * X_test.ix[:, -200:])
#X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])
#tmp = abs(X_test.ix[:, 2:202] - X_test.ix[:, -200:])
#X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])

In [77]:
tmp = abs(X_test.ix[:, 2:202] - X_test.ix[:, 202:402])
X_test = pd.concat([X_test, tmp], axis=1, join_axes=[X_test.index])

In [85]:
del X_train

In [12]:
#X_train = pd.read_csv("data/train_inter.csv")
q_train = pd.read_csv("data/freq_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

f_train = pd.read_csv("data/train_features.csv", encoding = "ISO-8859-1")
f_train = f_train.ix[:, 2:]
X_train = pd.concat([X_train, f_train], axis=1)

In [13]:
q_train = pd.read_csv("data/f1_train.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [14]:
q_train = pd.read_csv("data/train_lda_feat.csv", encoding = "ISO-8859-1")
q_train = q_train.ix[:, 6:]
X_train = pd.concat([X_train, q_train], axis=1)

In [15]:
q_train = pd.read_csv("stacking/nn_21.csv")
q_train[:5]

,lstm
0,0.588275
1,0.203615
2,0.476624
3,0.016573
4,0.004713


In [16]:
q_train = pd.read_csv("stacking/nn_21.csv")
X_train = pd.concat([X_train, q_train], axis=1)

In [33]:
X_train[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,kur_q2vec,q1_q2_intersect,cosine_distance_lda,cityblock_distance_lda,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda,lstm
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.337301,0,0.046653,0.232798,1.0,1.266667,0.137590,0.127564,0.134090,0.588275
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034444,0,0.748658,1.241746,1.0,5.400000,0.538923,0.423051,0.812442,0.203615


In [45]:
X_train[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,20,21,22,23,24,25,26,27,28,29
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.041053,0.105704,0.078461,-0.064659,-0.015350,0.129838,0.072293,-0.010824,0.005664,0.038002
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.118016,-0.041619,-0.026479,0.073949,0.089666,0.025895,0.008982,0.057382,0.008907,-0.046267


In [69]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,192,193,194,195,196,197,198,199,qid1_max_kcore,qid2_max_kcore
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,-0.164904,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [90]:
del tmp

NameError: name 'tmp' is not defined

In [41]:
import gc
gc.collect()

37

In [36]:
q_test = pd.read_csv("data/freq_test.csv")


In [37]:
X_test = pd.concat([X_test, q_test], axis=1)

In [38]:
f_test = pd.read_csv("data/test_features.csv", encoding = "ISO-8859-1")
f_test = f_test.ix[:, 2:]

In [39]:
X_test = pd.concat([X_test, f_test], axis=1)

In [40]:
q_test = pd.read_csv("data/f1_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [42]:
q_test = pd.read_csv("data/test_lda_feat.csv", encoding = "ISO-8859-1")
q_test = q_test.ix[:, 3:]
X_test = pd.concat([X_test, q_test], axis=1)

In [43]:
q_test = pd.read_csv("stacking/nn_21_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [23]:
X_test.shape

(2345796, 442)

In [29]:
m_svd.index = range(len(m_svd))

In [31]:
m_svd.shape

(2345796, 30)

In [32]:
X_test = pd.concat([X_test, m_svd], axis=1)

In [33]:
q_test = pd.read_csv("stacking/nn_21_test.csv")
X_test = pd.concat([X_test, q_test], axis=1)

In [44]:
X_test[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,kur_q2vec,q1_q2_intersect,cosine_distance_lda,cityblock_distance_lda,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda,lstm
0,0.266667,0.274019,0.0,0.0,0.0,0.0,0.0,-0.156312,0.0,0.0,...,-0.144866,0,0.573341,1.160190,1.0,7.833784,0.399663,0.290631,0.65838,0.102577
1,0.500000,0.480962,0.0,0.0,0.0,0.0,0.0,-0.164904,0.0,0.0,...,-0.042937,0,0.251020,0.615409,1.0,3.595264,0.265033,0.210156,0.35913,0.310506


In [35]:
len(X_test)

2345796

In [83]:
len(m_svd)

2345796

In [26]:
X_train[:2]

,word_match,tfidf_word_match,0,1,2,3,4,5,6,7,...,kur_q2vec,q1_q2_intersect,cosine_distance_lda,cityblock_distance_lda,jaccard_distance_lda,canberra_distance_lda,euclidean_distance_lda,minkowski_distance_lda,braycurtis_distance_lda,lstm
0,0.727273,0.772164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.337301,0,0.046653,0.232798,1.0,1.266667,0.137590,0.127564,0.134090,0.588275
1,0.307692,0.361758,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.034444,0,0.748658,1.241746,1.0,5.400000,0.538923,0.423051,0.812442,0.203615


In [ ]:
tmp = X_train['lstm']
del X_train['lstm']
X_train['lstm']

In [29]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 2000

#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.05,
    'min_child_weight': 50,
    #'max_depth': 7
    #'num_class': 3
    
}

pred_train = np.zeros(len(y_train))
xgbs = []
sc,sc_mean = [],[]

for itr, ite in folds:
    ypred = []
    for i in range(1):
               
        param['seed'] = i+1
        
        #xgboost
        '''
        Xdatatrain = xgb.DMatrix(data=X_train.ix[itr, :].values,
                                     label=y_train.ix[itr].values)
        Xdataval = xgb.DMatrix(data=X_train.ix[ite, :].values,
                                     label=y_train.ix[ite].values)

        plst = list(param.items())
        watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]
        bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)

        
        #rc = ensemble.ExtraTreesClassifier(n_estimators=1300, criterion='gini', max_depth=None, n_jobs=-1)
        #pred_train[ite] = bst.predict(Xdataval)
        #neigh = KNeighborsClassifier(n_neighbors=2)
        #neigh.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = bst.predict(Xdataval)
        #ypred = neigh.predict_proba(X_train.ix[ite, :])
        xgbs.append(bst)
        '''
        
        # train
        lgb = lgbm.sklearn.LGBMClassifier(n_estimators=2000, seed=0, **params)
        lgb.fit(X_train.ix[itr, :], y_train.ix[itr])
        ypred = lgb.predict_proba(X_train.ix[ite, :])[:, 1]

        '''
        model.fit(X_train.ix[itr, :].values, y_train_cat[itr],
            epochs=60,
            batch_size=1000)
        ypred = model.predict(X_train.ix[ite, :].values)
        '''
    #ypred = sum(ypred) / len(ypred) 0.401408 0.392476
    pred_train[ite] = ypred

    
    sc.append(log_loss(y_train.ix[ite, :], pred_train[ite]))

    
print('XGB: {:.3f} +- {:.3f}'.format(np.mean(sc), np.std(sc)))

/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected.

XGB: 0.217 +- 0.002


In [30]:
def foo(x):
    a = 0.165 / 0.37
    b = (1 - 0.165) / (1 - 0.37) 
    return  a * x / (a * x + b * (1 - x))

In [31]:
pred_train = foo(pred_train)

In [32]:
pred_train = pd.DataFrame(pred_train)
pred_train.columns = ['y']
pred_train.to_csv("stacking/lgb_23.csv", index=None)

In [58]:
del pred_train

In [45]:
import gc
gc.collect()

204

In [55]:
submit = pd.read_csv("data/sample_submission.csv")

In [56]:
#LGBM
params = {
    #'task': 'train',
    #'boosting_type': 'gbdt',
    #'objective': 'regression',
    #'metric': {'l2', 'auc'},
    #'num_leaves': 31,
    #'feature_fraction': 0.9,
    #'bagging_fraction': 0.8,
    #'bagging_freq': 5,
    #'subsample': 0.8,
    #'colsample_bytree': 0.7,
    'objective': 'binary',
    'nthread': -1,
    'learning_rate': 0.1,
    'min_child_weight': 100,
    #'max_depth': 6
    #'num_class': 3
    
}

In [34]:
lgb = lgbm.sklearn.LGBMClassifier(n_estimators=2000, seed=0, **params)
lgb.fit(X_train, y_train)


/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:112: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/fao3864/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:147: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', colsample_bytree=1, drop_rate=0.1,
        is_unbalance=False, learning_rate=0.05, max_bin=255, max_depth=-1,
        max_drop=50, min_child_samples=10, min_child_weight=50,
        min_split_gain=0, n_estimators=2000, nthread=-1, num_leaves=31,
        objective='binary', reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
        seed=0, sigmoid=1.0, silent=True, skip_drop=0.5, subsample=1,
        subsample_for_bin=50000, subsample_freq=1, uniform_drop=False,
        xgboost_dart_mode=False)

In [48]:
X_test = X_test.ix[1000001:, :]

In [49]:
test_pred2 = lgb.predict_proba(X_test)[:, 1]

In [47]:
test_pred1[:4]

array([  5.93303105e-03,   1.74597458e-01,   5.54921827e-01,
         4.47564805e-04])

In [127]:
test_pred1[-4:]

array([  4.85291907e-04,   2.61291438e-01,   6.26312391e-01,
         1.66797416e-03])

In [50]:
test_pred2[:4]

array([  1.25787340e-01,   2.73604227e-05,   6.46462160e-05,
         2.09556947e-02])

In [151]:
test_pred2[-4:]

array([  1.67587699e-03,   2.31711933e-04,   3.72899384e-01,
         3.89707027e-01])

In [148]:
len(test_pred1)

1000001

In [152]:
len(test_pred2)

1345795

In [51]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [52]:
len(test_pred)

2345796

In [53]:
test_pred

array([  5.93303105e-03,   1.74597458e-01,   5.54921827e-01, ...,
         1.01165868e-04,   4.72257462e-01,   5.25368347e-01])

In [146]:
del pred_train

NameError: name 'pred_train' is not defined

In [54]:
import gc
gc.collect()

74

In [38]:
param = {}
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'logloss'
param['max_depth'] = 7
# param['booster'] = 'dart'
#param['booster'] = 'gblinear'
param['eta'] = 0.1
#param['subsample'] = 1.0
# param["scale_pos_weight"] = 0.5
#param['colsample_bytree'] = 0.7
param['nthread'] = 8
#param['num_class'] =3
#param['alpha'] = 0.00
#param['lambda_bias'] = 0.1
# param['lambda'] = 0
#param['min_child_weight'] = 100

# if param['booster'] == 'gblinear': param['eta'] *= 1e-4
numround = 2000

In [39]:
Xdatatrain = xgb.DMatrix(data=X_train.ix[:, :].values,
                                     label=y_train.ix[:].values)

In [40]:
Xdataval = xgb.DMatrix(data=X_train.ix[:10, :].values,
                                     label=y_train.ix[:10].values)

In [62]:
del Xdatatest2

In [57]:
Xdatatest2 = xgb.DMatrix(data=X_test.ix[1000001:, :].values)

In [42]:

plst = list(param.items())
watchlist = [(Xdatatrain, 'train'), (Xdataval, 'eval')]

In [45]:
bst = xgb.train(plst, Xdatatrain, numround, evals=watchlist, verbose_eval=15)


[0]	train-logloss:0.633506	eval-logloss:0.646774
[15]	train-logloss:0.323969	eval-logloss:0.388133
[30]	train-logloss:0.271702	eval-logloss:0.343325
[45]	train-logloss:0.257076	eval-logloss:0.341127
[60]	train-logloss:0.248104	eval-logloss:0.316478
[75]	train-logloss:0.241793	eval-logloss:0.306781
[90]	train-logloss:0.236545	eval-logloss:0.294224
[105]	train-logloss:0.232073	eval-logloss:0.288613
[120]	train-logloss:0.228825	eval-logloss:0.282442
[135]	train-logloss:0.225883	eval-logloss:0.283732
[150]	train-logloss:0.222672	eval-logloss:0.279839
[165]	train-logloss:0.220304	eval-logloss:0.271811
[180]	train-logloss:0.218097	eval-logloss:0.271271
[195]	train-logloss:0.215673	eval-logloss:0.269905
[210]	train-logloss:0.213894	eval-logloss:0.270819
[225]	train-logloss:0.211979	eval-logloss:0.273126
[240]	train-logloss:0.210103	eval-logloss:0.267782
[255]	train-logloss:0.208528	eval-logloss:0.267423
[270]	train-logloss:0.207038	eval-logloss:0.269102
[285]	train-logloss:0.205572	eval-loglo

In [61]:
test_pred2 = bst.predict(Xdatatest2)


In [54]:
test_pred1[:4]

array([  4.93649067e-03,   1.69867009e-01,   5.44634938e-01,
         2.05250995e-04], dtype=float32)

In [50]:
len(test_pred1)

1000001

In [53]:
test_pred1[:4]

array([  2.29527755e-03,   5.45756638e-01,   6.37812853e-01,
         4.23383579e-04], dtype=float32)

In [64]:
test_pred = np.concatenate([test_pred1, test_pred2], axis=0)

In [65]:
len(test_pred)

2345796

In [56]:
test_pred = foo(test_pred)

In [57]:
submit['is_duplicate'] = test_pred

In [58]:
submit.to_csv("stacking/lgb_23_test.csv", index=None)

In [13]:
submit

,test_id,is_duplicate
0,0,0.020263
1,1,0.196667
2,2,0.327514
3,3,0.001061
4,4,0.464111
5,5,0.009520
6,6,0.393998
7,7,0.838837
8,8,0.326061
9,9,0.004459


In [59]:
submit

,test_id,is_duplicate
0,0,0.002004
1,1,0.066443
2,2,0.295527
3,3,0.000151
4,4,0.111192
5,5,0.001435
6,6,0.998418
7,7,0.484766
8,8,0.313561
9,9,0.000257
